In [ ]:
pip install codeflare-sdk codeflare-torchx

In [1]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication, RayJobClient, Authentication
import os
import sys

In [2]:
# Authenticate the CodeFlare SDK
# On OpenShift, you can retrieve the token by running `oc whoami -t`,
# and the server with `oc cluster-info`.
auth = TokenAuthentication(
    token = 'sha256~fXiyTta4BqDo6fpQf1VRAJ60LABQtz2p0blvpORnfuE',
    server = 'https://api.rosa-wswzx.e1i3.p3.openshiftapps.com:443',
    skip_tls=False
)
auth.login()

'Logged into https://api.rosa-wswzx.e1i3.p3.openshiftapps.com:443'

In [3]:
# Configure the Ray cluster
cluster = Cluster(ClusterConfiguration(
    head_cpu_requests=10,
    head_cpu_limits=12,
    head_memory_requests=10,
    head_memory_limits=12,
    name='dd-rl',
    namespace='hello-chris-ai-4',
    num_workers=3,
    worker_cpu_requests=12,
    worker_cpu_limits=16,
    worker_memory_requests=12,
    worker_memory_limits=20,
    # Use the following parameters with NVIDIA GPUs
    image="quay.io/cnuland/dd-kuberay-worker:latest",
    head_extended_resource_requests={'nvidia.com/gpu':1},
    worker_extended_resource_requests={'nvidia.com/gpu':1},
    write_to_file=False,
    local_queue="user-queue"
))

Yaml resources loaded for dd-rl


Output()

In [4]:
# Create the Ray cluster
cluster.up()

Ray Cluster: 'dd-rl' has successfully been created


In [5]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [6]:
cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-4.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=325075;https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                       Cluster Resources                             │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮         │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │         │ 
 │   │             │  │                                      │         │ 
 │   │  3          │  │  12G~20G     12~16       1           │         │ 
 │   │             │  │                                      │         │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯         │ 
 ╰─────────────────────────────────────────────────────────────────────╯

RayCluster(name='dd-rl', status=<RayClusterStatus.READY: 'ready'>, head_cpu_requests=10, head_cpu_limits=12, head_mem_requests='10G', head_mem_limits='12G', num_workers=3, worker_mem_requests='12G', worker_mem_limits='20G', worker_cpu_requests=12, worker_cpu_limits=16, namespace='hello-chris-ai-4', dashboard='https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com', worker_extended_resources={'nvidia.com/gpu': 1}, head_extended_resources={'nvidia.com/gpu': 1})

In [7]:
clusterDetails = cluster.details()

                     🚀 CodeFlare Cluster Details 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-4.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=918366;https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 │                       Cluster Resources                             │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮         │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │         │ 
 │   │             │  │                                      │         │ 
 │   │  3          │  │  12G~20G     12~16       1           │         │ 
 │   │             │  │                                      │         │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯         │ 
 ╰─────────────────────────────────────────────────────────────────────╯

In [9]:
# Create or connect to the Ray cluster
cluster.status()
print(f"Ray Dashboard URL: {clusterDetails.dashboard}")


# Initialize RayJobClient
# The address for RayJobClient should be the Ray head service, often exposed via a route or service URL.
# CodeFlare SDK's `cluster.details()` might provide this, or `cluster.cluster_uri()` for internal access.
# For external access (like from your notebook to submit job), you might need the Ray dashboard or a client service URL.
# Let's assume client can connect to where jobs are submitted (usually Ray head/dashboard)
# If your notebook runs outside the OpenShift cluster, you might need an external route to Ray head.
# For simplicity, using cluster_uri() which is often for internal connections; adapt if needed.
# A common pattern is to use the dashboard URL for the RayJobClient if it's accessible.
try:
    # Attempt connection to Ray head, may need adjustment based on network setup
    # For jobs, it's often `address="ray://<head_service_host>:<port>"`
    # If dashboard_url is http://...:8265, client address is often the same host but port 10001 for ray client.
    # For KubeRay, the service for Ray client is often 'raycluster-kuberay-head-svc'.
    # The dashboard_url() from codeflare might be what you need directly or with minor modification.
    
    # A common approach is to use the dashboard URL if it's correctly pointing to the Ray head service endpoint.
    # However, RayJobClient expects a Ray Client server address (ray://) or HTTP proxy.
    # If your 'cluster.dashboard_url()' provides an HTTP URL for the dashboard,
    # Ray Client can connect via HTTP Proxy if enabled (Ray >= 1.13).
    # ray_client_address = "http://" + cluster.dashboard_url().split("://")[1] # Simplified, assumes standard port or proxy setup
    
    # It's safer to get the specific client connection string if available.
    # Let's assume Codeflare SDK handles this abstraction or you can derive it.
    # If running locally and port-forwarding Ray head's client port (default 10001):
    # client_address = "ray://localhost:10001"
    # When submitting from the environment that can reach the Ray head:
    #client_address = clusterDetails.dashboard # Check if this works directly or needs adjustment for RayJobClient
                                         # It may need to be like "http://<host>:<dashboard_port>"
    client_address = clusterDetails.dashboard
    print(f"Attempting to connect RayJobClient to: {client_address}")
    client = cluster.job_client


except Exception as e:
    print(f"Error initializing RayJobClient: {e}")
    print("Please ensure the Ray client server is accessible from where you are running this notebook.")
    print("You might need to set up port-forwarding or use an external route to the Ray head service.")
    client = None # Ensure client is None if connection fails

    
if client:
    print("Submitting Ray job...")

    try:
        submission_id = client.submit_job(
            entrypoint="python run-ray-dd.py",
            runtime_env={
                "env_vars": {
                    'S3_ACCESS_KEY_ID': "minio",
                    'S3_SECRET_ACCESS_KEY': "minio123",
                    'S3_REGION_NAME': "region",
                    'S3_ENDPOINT_URL': "https://minio-s3-minio.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com",
                     'S3_BUCKET_NAME': "sessions",

                },
            
                'working_dir': './',
                'pip': ["boto3"],
                "excludes": ["*.sh", "*.ipynb", "*.md"],
            }
        )
        print(f"Job submitted successfully! Submission ID: {submission_id}")
        print("You can monitor the job status using client.get_job_status(submission_id)")
        print("And logs using client.get_job_logs(submission_id)")

        # Example: Poll for job status
        # import time
        while True:
            status = client.get_job_status(submission_id)
            print(f"Job status: {status}")
            if status in [JobStatus.SUCCEEDED, JobStatus.FAILED, JobStatus.STOPPED]:
                break
            time.sleep(5)
            print(f"Final job logs:\n{client.get_job_logs(submission_id)}")

    except Exception as e:
        print(f"Error submitting job: {e}")
else:
    print("RayJobClient not initialized. Cannot submit job.")

                      🚀 CodeFlare Cluster Status 🚀                     
                                                                         
 ╭─────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                              │ 
 │   dd-rl                                                 Active ✅   │ 
 │                                                                     │ 
 │   URI: ray://dd-rl-head-svc.hello-chris-ai-4.svc:10001              │ 
 │                                                                     │ 
 │   ]8;id=212736;https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com\Dashboard🔗]8;;\                                                       │ 
 │                                                                     │ 
 ╰─────────────────────────────────────────────────────────────────────╯

2025-05-19 04:09:31,851	INFO dashboard_sdk.py:385 -- Package gcs://_ray_pkg_4f1acf0e28a879c6.zip already exists, skipping upload.


Ray Dashboard URL: https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com
Attempting to connect RayJobClient to: https://ray-dashboard-dd-rl-hello-chris-ai-4.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com
Submitting Ray job...
Job submitted successfully! Submission ID: raysubmit_f9td1ftWyhqMecNv
You can monitor the job status using client.get_job_status(submission_id)
And logs using client.get_job_logs(submission_id)
Job status: PENDING
Error submitting job: name 'JobStatus' is not defined


In [10]:
client.list_jobs()

[JobDetails(type=<JobType.SUBMISSION: 'SUBMISSION'>, job_id='03000000', submission_id='raysubmit_7r6QXiEVujAMsNBS', driver_info=DriverInfo(id='03000000', node_ip_address='10.129.9.160', pid='1796'), status=<JobStatus.RUNNING: 'RUNNING'>, entrypoint='python run-ray-dd.py', message='Job is currently running.', error_type=None, start_time=1747627451820, end_time=None, metadata={}, runtime_env={'working_dir': 'gcs://_ray_pkg_4f1acf0e28a879c6.zip', 'excludes': ['*.sh', '*.ipynb', '*.md'], 'pip': {'packages': ['boto3'], 'pip_check': False}, 'env_vars': {'S3_ACCESS_KEY_ID': 'minio', 'S3_SECRET_ACCESS_KEY': 'minio123', 'S3_REGION_NAME': 'region', 'S3_ENDPOINT_URL': 'https://minio-s3-minio.apps.rosa.rosa-wswzx.e1i3.p3.openshiftapps.com', 'S3_BUCKET_NAME': 'sessions'}, '_ray_commit': 'daca7b2b1a950dc7f731e34e74c76ae383794ffe'}, driver_agent_http_address='http://10.129.9.160:52365', driver_node_id='d5eb85c43e617cddf730030b6862c146e990ed05c206c9f36346a474', driver_exit_code=None),
 JobDetails(type

In [11]:
client.stop_job(submission_id)

True

In [ ]:
cluster.down()